In [1]:
import csv
import requests
import xml.etree.ElementTree as ET

In [2]:
def parseXML(xmlfile):
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    newsitems = []
        
    for drug in root.findall('{http://www.drugbank.ca}drug'):
        news = {}
        for drugAttri in drug:
            if drugAttri.tag == '{http://www.drugbank.ca}drugbank-id' and drugAttri.get('primary', default = False):
                news['drug id'] = drugAttri.text
            elif drugAttri.tag == '{http://www.drugbank.ca}name':
                news['name'] = drugAttri.text
            elif drugAttri.tag == '{http://www.drugbank.ca}calculated-properties':
                for drugProperty in drugAttri.findall('{http://www.drugbank.ca}property'):
                    find = False
                    for drugKind in drugProperty.findall('{http://www.drugbank.ca}kind'):
                        if drugKind.text == 'SMILES':
                            find = True
                    if find == True:
                        for drugValue in drugProperty.findall('{http://www.drugbank.ca}value'):
                            news['smiles'] = drugValue.text
        newsitems.append(news)
    return newsitems

In [3]:
def savetoCSV(newsitems, filename):
    fields = ['drug id', 'name', 'smiles']
    
    with open(filename, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = fields)
        writer.writeheader()
        writer.writerows(newsitems)

In [4]:
def main():
    newsitems = parseXML('drugbank.xml')
    savetoCSV(newsitems, 'output.csv')

if __name__ == "__main__":
    main()